# Minimum Eigen Optimizer


Finding the solution to a Quadratic Unconstrained Binary Optimization(QUBO) problem is equivalent in finding the ground state of a corresponding Ising Hamiltonian - applicable to optimization, quantum chemistry and physics.
The binary variables {0,1} are replaced by spin variables with values {−1,+1}, which allows to replace the resulting spin variables by Pauli Z matrices, and thus, an Ising Hamiltonian. 
Details on this mapping -> [1].

Qiskit provides automatic conversion from a suitable QuadraticProgram to an Ising Hamiltonian, which then allows to leverage all the MinimumEigenSolver such as - VQE, - QAOA, or - NumpyMinimumEigensolver(classical exact method).

Qiskit wraps the translation to an Ising Hamiltonian (in Qiskit Aqua also called Operator), the call to an MinimumEigensolver as well as the translation of the results back to OptimizationResult in the MinimumEigenOptimizer.

In the following we first illustrate the conversion from a QuadraticProgram to an Operator and then show how to use the MinimumEigenOptimizer with different MinimumEigensolver to solve a given QuadraticProgram. The algorithms in Qiskit automatically try to convert a given problem to the supported problem class if possible, for instance, the MinimumEigenOptimizer will automatically translate integer variables to binary variables or add a linear equality constraints as a quadratic penalty term to the objective. It should be mentioned that Aqua will through a QiskitOptimizationError if conversion of a quadratic program with integer variable is attempted.

The circuit depth of QAOA potentially has to be increased with the problem size, which might be prohibitive for near-term quantum devices. A possible workaround is Recursive QAOA, as introduced in [2]. Qiskit generalizes this concept to the RecursiveMinimumEigenOptimizer, which is introduced at the end of this tutorial.
References

[1] [A. Lucas, Ising formulations of many NP problems, Front. Phys., 12 (2014).](https://arxiv.org/abs/1302.5843)

[2] [S. Bravyi, A. Kliesch, R. Koenig, E. Tang, Obstacles to State Preparation and Variational Optimization from Symmetry Protection, arXiv preprint arXiv:1910.08980 (2019).](https://arxiv.org/abs/1910.08980)


## Converting a QUBO to an Operator

In [1]:
from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

In [2]:
# create a QUBO
qubo = QuadraticProgram()
qubo.binary_var('x')
qubo.binary_var('y')
qubo.binary_var('z')
qubo.minimize(linear=[1,-2,3], quadratic={('x', 'y'): 1, ('x', 'z'): -1, ('y', 'z'): 2})
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



In [3]:
#Next we translate this QUBO into an Ising operator. 
#This results not only in an Operator but also in a constant offset to be taking into account 
#to shift the resulting value.

op, offset = qubo.to_ising()
print('offset: {}'.format(offset))
print('operator:')
print(op)

offset: 1.5
operator:
SummedOp([
  -0.5 * IIZ,
  0.25 * IZI,
  -1.75 * ZII,
  0.25 * IZZ,
  -0.25 * ZIZ,
  0.5 * ZZI
])


In [4]:
#Sometimes a QuadraticProgram might also directly be given in the form of an Operator. 
#For such cases, Qiskit also provides a converter from an Operator back to a QuadraticProgram, 
#which we illustrate in the following.

qp=QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x_0 - 2 x_1 + 3 x_2 + [ 2 x_0*x_1 - 2 x_0*x_2 + 4 x_1*x_2 ]/2
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1

Binaries
 x_0 x_1 x_2
End



In [5]:
#This converter allows to translate an Operator to a QuadraticProgram and 
#then solve the problem with other algorithms that are not based on the Ising Hamiltonian representation, 
#such as the GroverOptimizer.

## Solving a QUBO with the MinimumEigenOptimizer

In [7]:
#We start by initializing the MinimumEigensolver we want to use.
qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator'))
exact_mes = NumPyMinimumEigensolver()

#Then, we use the MinimumEigensolver to create MinimumEigenOptimizer.
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [8]:
#We first use the MinimumEigenOptimizer based on the classical exact 
#NumPyMinimumEigensolver to get the optimal benchmark solution for this small example.
exact_result = exact.solve(qubo)
print(exact_result)

optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


In [9]:
#Next we apply the MinimumEigenOptimizer based on QAOA to the same problem.
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


## RecursiveMinimumEigenOptimizer

In [11]:
#The RecursiveMinimumEigenOptimizer takes a MinimumEigenOptimizer as input and 
#applies the recursive optimization scheme to reduce the size of the problem one variable at a time. 
#Once the size of the generated intermediate problem is below a given threshold (min_num_vars), 
#the RecursiveMinimumEigenOptimizer uses another solver (min_num_vars_optimizer), 
#e.g., an exact classical solver such as CPLEX or the MinimumEigenOptimizer based on the NumPyMinimumEigensolver.

#In the following, we show how to use the RecursiveMinimumEigenOptimizer 
#using the two MinimumEigenOptimizer introduced before.

#First, we construct the RecursiveMinimumEigenOptimizer 
#such that it reduces the problem size from 3 variables to 1 variable and 
#then uses the exact solver for the last variable. 
#Then we call solve to optimize the considered problem.

rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=1, min_num_vars_optimizer=exact)

rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)



optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS
